<a href="https://colab.research.google.com/github/REVOLUTTION/TelecomX_parte2/blob/main/telecomx_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
datos = pd.read_csv('/content/datos_tratados.csv')
datos

,id_cliente,tasa_abandono,genero,adulto_mayor,tiene_pareja,tiene_dependientes,meses_contrato,servicio_telefono,lineas_adicionales,servicio_internet,...,proteccion_dispositivos,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,facturacion_sin_papel,metodo_pago,facturacion_mensual,facturacion_total,facturacion_diaria
0,0002-ORFBO,0.0,Female,0,1,1,9,1,No,DSL,...,No,Yes,Yes,No,One year,1,Mailed check,65.60,593.30,2.186667
1,0003-MKNFE,0.0,Male,0,0,0,9,1,Yes,DSL,...,No,No,No,Yes,Month-to-month,0,Mailed check,59.90,542.40,1.996667
2,0004-TLHLJ,1.0,Male,0,0,0,4,1,No,Fiber optic,...,Yes,No,No,No,Month-to-month,1,Electronic check,73.90,280.85,2.463333
3,0011-IGKFF,1.0,Male,1,1,0,13,1,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.00,1237.85,3.266667
4,0013-EXCHZ,1.0,Female,1,1,0,3,1,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.90,267.40,2.796667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,0.0,Female,0,0,0,13,1,No,DSL,...,No,Yes,No,No,One year,0,Mailed check,55.15,742.90,1.838333
7263,9992-RRAMN,1.0,Male,0,1,0,22,1,Yes,Fiber optic,...,No,No,No,Yes,Month-to-month,1,Electronic check,85.10,1873.70,2.836667
7264,9992-UJOEL,0.0,Male,0,0,0,2,1,No,DSL,...,No,No,No,No,Month-to-month,1,Mailed check,50.30,92.75,1.676667
7265,9993-LHIEB,0.0,Male,0,1,1,67,1,No,DSL,...,Yes,Yes,No,Yes,Two year,0,Mailed check,67.85,4627.65,2.261667


In [ ]:
datos.describe()

,tasa_abandono,adulto_mayor,tiene_pareja,tiene_dependientes,meses_contrato,servicio_telefono,facturacion_sin_papel,facturacion_mensual,facturacion_total,facturacion_diaria
count,7043.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000
mean,0.265370,0.162653,0.484106,0.300124,32.346498,0.902711,0.593230,64.720098,2277.182035,2.157337
std,0.441561,0.369074,0.499782,0.458343,24.571773,0.296371,0.491265,30.129572,2268.648587,1.004319
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000,0.608333
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.425000,396.200000,1.180833
50%,0.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.300000,1389.200000,2.343333
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.875000,3778.525000,2.995833
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,3.958333


In [ ]:
datos.dtypes

,0
id_cliente,object
tasa_abandono,float64
genero,object
adulto_mayor,int64
tiene_pareja,int64
tiene_dependientes,int64
meses_contrato,int64
servicio_telefono,int64
lineas_adicionales,object
servicio_internet,object


In [ ]:
# Eliminar ID
datos = datos.drop(columns=["id_cliente"])

In [ ]:
# Detectar columnas categóricas
cat_cols = datos.select_dtypes(include=["object"]).columns.tolist()

In [ ]:
# Crear transformador One-Hot
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore"), cat_cols)
    ],
    remainder="passthrough"  # deja columnas numéricas sin cambios
)

# Aplicar transformación
X_encoded = preprocessor.fit_transform(datos)

# Obtener nombres de columnas finales
cat_features = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
num_features = datos.select_dtypes(exclude=["object"]).columns.tolist()
all_features = list(num_features) + list(cat_features)

# Convertir a DataFrame
datos_codificados = pd.DataFrame(X_encoded, columns=all_features, index=datos.index)

# Mostrar primeras filas del dataset transformado
print("\n=== Dataset transformado a numérico (One-Hot Encoding) ===")
print(datos_codificados.head())



=== Dataset transformado a numérico (One-Hot Encoding) ===
   tasa_abandono  adulto_mayor  tiene_pareja  tiene_dependientes  \
0            0.0           0.0           0.0                 0.0   
1            1.0           0.0           1.0                 0.0   
2            1.0           0.0           0.0                 1.0   
3            1.0           0.0           0.0                 1.0   
4            0.0           0.0           0.0                 1.0   

   meses_contrato  servicio_telefono  facturacion_sin_papel  \
0             0.0                0.0                    0.0   
1             0.0                0.0                    0.0   
2             0.0                0.0                    0.0   
3             0.0                0.0                    0.0   
4             0.0                0.0                    0.0   

   facturacion_mensual  facturacion_total  facturacion_diaria  ...  \
0                  0.0                1.0                 0.0  ...   
1           

In [ ]:
# Contar cuántos clientes cancelaron y cuántos permanecieron
proporcion = datos['tasa_abandono'].value_counts(normalize=True)

print("Proporción de clientes por clase:")
print(proporcion)

Proporción de clientes por clase:
tasa_abandono
0.0    0.73463
1.0    0.26537
Name: proportion, dtype: float64


In [ ]:
print("\nEn porcentaje:")
print(proporcion * 100)


En porcentaje:
tasa_abandono
0.0    73.463013
1.0    26.536987
Name: proportion, dtype: float64
